In [69]:
import os
import socket
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, length, when, col
from pyspark.sql.types import BooleanType, IntegerType, LongType, StringType, ArrayType, FloatType, StructType, StructField
from pyspark.sql.functions import *
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType
from jinja2 import Environment, FileSystemLoader

In [70]:
# setting constants
APP_NAME = "VVORONIN-SPARK-APP_send"
NORMALIZED_APP_NAME = APP_NAME.replace('/', '_').replace(':', '_')

APPS_TMP_DIR = os.path.join(os.getcwd(), "tmp")
APPS_CONF_DIR = os.path.join(os.getcwd(), "conf")
APPS_LOGS_DIR = os.path.join(os.getcwd(), "logs")
LOG4J_PROP_FILE = os.path.join(APPS_CONF_DIR, "pyspark-log4j-{}.properties".format(NORMALIZED_APP_NAME))
LOG_FILE = os.path.join(APPS_LOGS_DIR, 'pyspark-{}.log'.format(NORMALIZED_APP_NAME))
EXTRA_JAVA_OPTIONS = "-Dlog4j.configuration=file://{} -Dspark.hadoop.dfs.replication=1 -Dhttps.protocols=TLSv1.0,TLSv1.1,TLSv1.2,TLSv1.3"\
    .format(LOG4J_PROP_FILE)

LOCAL_IP = socket.gethostbyname(socket.gethostname())

In [71]:
# preparing configuration files from templates
for directory in [APPS_CONF_DIR, APPS_LOGS_DIR, APPS_TMP_DIR]:
    if not os.path.exists(directory):
        os.makedirs(directory)

env = Environment(loader=FileSystemLoader('/opt'))
template = env.get_template("pyspark_log4j.properties.template")
template\
    .stream(logfile=LOG_FILE)\
    .dump(LOG4J_PROP_FILE)

In [72]:
# spark = SparkSession\
#     .builder\
#     .appName(APP_NAME)\
#     .master("k8s://https://10.32.7.103:6443")\
#     .config("spark.driver.host", LOCAL_IP)\
#     .config("spark.driver.bindAddress", "0.0.0.0")\
#     .config("spark.executor.instances", "3")\
#     .config("spark.executor.cores", '2')\
#     .config("spark.memory.fraction", "0.8")\
#     .config("spark.memory.storageFraction", "0.6")\
#     .config("spark.executor.memory", "4g")\
#     .config("spark.driver.extraJavaOptions", EXTRA_JAVA_OPTIONS)\
#     .config("spark.kubernetes.namespace", "{{user}}")\
#     .config("spark.kubernetes.driver.label.appname", APP_NAME)\
#     .config("spark.kubernetes.executor.label.appname", APP_NAME)\
#     .config("spark.kubernetes.container.image.pullPolicy", "Always")\
#     .config("spark.kubernetes.container.image", "node03.st:5000/spark-executor:{{user}}")\
#     .config("spark.local.dir", "/tmp/spark")\
#     .getOrCreate()

SPARK_ADDRESS = "k8s://https://10.32.7.103:6443"

spark = SparkSession\
    .builder\
    .appName("VVORONIN-SPARK-APP")\
    .master(SPARK_ADDRESS)\
    .config("spark.ui.port", "4040")\
    .config("spark.memory.fraction", "0.8")\
    .config("spark.memory.storageFraction", "0.6")\
    .config("spark.driver.memory", "4g")\
    .config("spark.driver.extraJavaOptions", EXTRA_JAVA_OPTIONS)\
    .config("spark.executor.memory", "6g")\
    .getOrCreate()

In [85]:
#для каждого поста получим его id его автора
posts_data = spark.read.json("hdfs:///shared/bigdata20/followers_posts_api_final.json")
posts_df = posts_data.where((col("text") != "") & (col("text") != " "))\
                .select(col("owner_id").alias("user_id"), "date", "text")

In [86]:
posts_df.toPandas().head()

,user_id,date,text
0,87449,1550165023,"Я люблю Вас. Я вчера, даже, хотел Вам это сказ..."
1,87449,1553774858,call me by your name ♡
2,87449,1555602008,🦋
3,87449,1558105050,"Браво,Юра !"
4,87449,1559301729,🕊


In [83]:
#для каждого id получим возраст пользовалетя и пол
followers_data = spark.read.json("hdfs:///shared/bigdata20/followers_info.json")
followers_df = followers_data.select(col("id").alias("user_id"), "sex", "bdate")\
                    .withColumn("month_between", 
                        months_between(
                            current_date(), 
                            concat_ws(
                                "-", regexp_extract(col("bdate"), r"(\d{1,2})\.(\d{1,2})\.(\d{4})", 3).cast("int"), 
                                regexp_extract(col("bdate"), r"(\d{1,2})\.(\d{1,2})\.(\d{4})", 2).cast("int"), 
                                regexp_extract(col("bdate"), r"(\d{1,2})\.(\d{1,2})\.(\d{4})", 1).cast("int")
                            ).cast("date")))\
                    .select("user_id", "sex", (col("month_between")/12).cast("int").alias("age"))\
                    .where(col("sex").isNotNull() & col("age").isNotNull())

In [84]:
followers_df.toPandas().head()

,user_id,sex,age
0,243,1,34
1,533,2,32
2,637,1,36
3,751,2,33
4,899,1,33


In [94]:
#соединим одно с другим
posts_with_user_info_df = posts_df.join(followers_df, ["user_id"])\
                            .select("user_id", col("date"),
                                    "text", (when(col("sex") == 1, "M").otherwise("F")).alias("sex"), "age")


In [95]:
posts_with_user_info_df.toPandas().head()

,user_id,date,text,sex,age
0,94494,1559647730,"Тот неловкий момент, когда прилетел из унылого...",F,33
1,94494,1559854126,Белые ночи + жаркие дни = ужин на речке под от...,F,33
2,94494,1560680534,Не велосипедом единым 🚣,F,33
3,94494,1560987374,Тот самый французский катамаран Energy Observe...,F,33
4,94494,1561155082,"Да, знатное шоу сегодня погода в Петербурге ра...",F,33


Начинаем передачу данных через kafka

In [105]:
from kafka import KafkaProducer
from tqdm import tqdm
import json
import time
import random
#from playsound import playsound

In [111]:
producer = KafkaProducer(bootstrap_servers="kafka-svc:9092", value_serializer=str.encode)
data = tqdm(posts_with_user_info_df.orderBy("date").rdd.toLocalIterator(), total=posts_with_user_info_df.count())

  0%|          | 0/73624 [00:00<?, ?it/s]

In [ ]:
for row in data:
    d = row.asDict()
    print(f"Sending a post from user id: {d['user_id']}, posted on: {d['date']}, sex - {d['sex']}")
    
    producer.send("main", json.dumps(d, ensure_ascii=False))
    producer.flush()
    time.sleep(random.random())

#playsound('~/nfs-home/beep-07a.wav')  
print("Broadcast has been finished")


Sending a post from user id: 273486249, posted on: 1546290007, sex - F


  0%|          | 1/73624 [02:45<3384:29:43, 165.49s/it]

Sending a post from user id: 30697, posted on: 1546290009, sex - F


  0%|          | 2/73624 [02:46<1403:53:14, 68.65s/it] 

Sending a post from user id: 127675121, posted on: 1546290040, sex - F


...

Sending a post from user id: 5550605, posted on: 1546464584, sex - F


  1%|          | 573/73624 [07:43<12:38:08,  1.61it/s]

Sending a post from user id: 3680017, posted on: 1546465293, sex - M


  1%|          | 574/73624 [07:43<12:19:15,  1.65it/s]

Sending a post from user id: 327458, posted on: 1546466205, sex - F
